# Decision Tree Implementation

### PreProcess Data

In [1]:
import PreProcessData as pre
import imp
imp.reload(pre)
X_train, X_valid, y_train, y_valid = pre.PreProcessData()
# print(X_train)

/tmp/ipykernel_33999/229690563.py:2: DeprecationWarning: the imp module is deprecated in favour of importlib and slated for removal in Python 3.12; see the module's documentation for alternative uses
  import imp


### Class Construction

In [5]:
import numpy as np
import pandas as pd

class DecisionTree:
    def __init__(self):
        self.tree = None
    
    def fit(self, X, y):
        self.tree = self.build_tree(X, y)

    def build_tree(self, X, y):
        # Base case: if all labels are the saame, return a leaf node
        if len(np.unique(y)) == 1:
            return {'label': y[0]}
        
        # Find the best split
        best_split = self.find_best_split(X, y)

        # Base case: if no split is found, return a leaf node with the majoirty label
        if best_split is None:
            return {'label': np.bincount(y).argmax()}

        # Split the data based on the best split
        left_idxs = X[:, best_split['feature']] <= best_split['threshold']
        right_idxs = ~left_idxs

        # Recursively build left and right subtrees
        left_tree = self.build_tree(X[left_idxs], y[left_idxs])
        right_tree = self.build_tree(X[right_idxs], y[right_idxs])

        # Return the node with the best split and its children
        return {'feature': best_split['feature'],
                'threshold': best_split['threshold'],
                'left': left_tree,
                'right': right_tree}

    def find_best_split(self, X, y):
        best_gini = float('inf')
        best_split = None

        for feature in range(X.shape[1]):
            unique_values = np.unique(X[:, feature])
            for threshold in unique_values:
                left_idxs = X[:, feature] <= threshold
                right_idxs = ~left_idxs

                gini = self.calculate_gini(y[left_idxs], y[right_idxs])
                if gini < best_gini:
                    best_gini = gini
                    best_split = {'feature': feature, 'threshold': threshold}

        return best_split

    def calculate_gini(self, left_labels, right_labels):
        total_samples = len(left_labels) + len(right_labels)
        gini_left = 1 - sum((np.bincount(left_labels) / len(left_labels)) ** 2)
        gini_right = 1 - sum((np.bincount(right_labels) / len(right_labels)) ** 2)
        weighted_gini = (len(left_labels) / total_samples) * gini_left + (len(right_labels) / total_samples) * gini_right
        return weighted_gini

    def predict(self, X):
        return np.array([self.predict_sample(x, self.tree) for x in X])

    def predict_sample(self, x, node):
        if 'label' in node:
            return node['label']
        
        if x[node['feature']] <= node['threshold']:
            return self.predict_sample(x, node['left'])
        else:
            return self.predict_sample(x, node['right'])

    def validate(self, y_pred, y_valid):
        num_of_correct_predictions = np.sum((y_pred >= 0.5) == y_valid)
        accuracy = num_of_correct_predictions / len(y_valid)
        return accuracy

In [6]:
tree = DecisionTree()
tree.fit(np.array(X_train), np.array(y_train))
predictions = tree.predict(np.array(X_valid))

RecursionError: maximum recursion depth exceeded while calling a Python object

In [ ]:
print(tree.validate(np.array(predictions), np.array(y_valid)))